# NER exercises

This is a modified, more compact version of (one variant of) the CRF tagger from the NER notebook. You may wish to use this as a starting point for doing some of the exercises.

(This first bit is Jupyter magic and required imports, feel free to ignore)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%run ../lib/conllu.ipynb
%run ../lib/visualization.ipynb

import sklearn_crfsuite
import eli5

from collections import namedtuple

from sklearn_crfsuite.metrics import flat_f1_score as f1_score
from sklearn_crfsuite.metrics import flat_classification_report as classification_report
from eli5.sklearn_crfsuite.explain_weights import sorted_for_ner

# Data loading

Look here if you want to work on English.

In [2]:
# Finnish

train_sentences = read_conll('../data/finer-featurized/digitoday.2014.train.conllu')
devel_sentences = read_conll('../data/finer-featurized/digitoday.2014.dev.conllu')
test_sentences = read_conll('../data/finer-featurized/digitoday.2015.test.conllu')
test_sentences_wiki = read_conll('../data/finer-featurized/wikipedia.test.conllu')

# English (uncomment if preferred)

# train_sentences = read_conll('../data/conll03-featurized/eng.train.conllu')
# devel_sentences = read_conll('../data/conll03-featurized/eng.dev.conllu')
# test_sentences = read_conll('../data/conll03-featurized/eng.test.conllu')
# test_sentences_wiki = []    # Sorry, we don't this dataset for English

## Data mangling and featurization

You'll want to edit this bit if you want to try different features.

In [3]:
Word = namedtuple('Word', [
    'form',
    'lemma',
    'upos',
    'xpos',
    'morpho',
])


def reformat_sentences(sentences):
    # separate tags, drop parse fields, cast as namedtuples
    tags, data = [], []
    for sentence in sentences:
        tags.append([w[0] for w in sentence])
        data.append([Word(*w[1:6]) for w in sentence])
    return tags, data


def less_minimal_featurizer(sentence, idx):
    word = sentence[idx]
    features = {
        'form': word.form,
        'form.lower()': word.form.lower(),
        'form[:2]': word.form[:2],
        'form[:3]': word.form[:3],
        'form[-2:]': word.form[-2:],
        'form[-3:]': word.form[-3:],
        'lemma': word.lemma,
        'lemma.lower()': word.lemma.lower(),
        'upos': word.upos,
        'xpos': word.xpos,
    }
    # morpho is either empty ("_") or like this: "Case=Nom|Number=Sing"
    if word.morpho == '_':
        features['morpho'] = '_'
    else:
        for attr, val in [m.split('=') for m in word.morpho.split('|')]:
            features['morpho.{}'.format(attr)] = val
    return features


def featurize_sentence(sentence, featurizer=less_minimal_featurizer):
    return [featurizer(sentence, i) for i in range(len(sentence))]


train_y, train_data = reformat_sentences(train_sentences)
devel_y, devel_data = reformat_sentences(devel_sentences)
test_y, test_data = reformat_sentences(test_sentences)
wiki_y, wiki_data = reformat_sentences(test_sentences_wiki)

train_x = [featurize_sentence(s) for s in train_data]
devel_x = [featurize_sentence(s) for s in devel_data]
test_x = [featurize_sentence(s) for s in test_data]
wiki_x = [featurize_sentence(s) for s in wiki_data]

## CRF training and prediction

This is the place to try different hyperparameters. Note that some of the parameters for `CRF()` have been filled in with their default values for reference.

In [4]:
%%time
crf = sklearn_crfsuite.CRF(all_possible_transitions=True)
crf.fit(train_x, train_y)
pred_y = crf.predict(devel_x)

CPU times: user 1min 22s, sys: 876 ms, total: 1min 23s
Wall time: 1min 23s


## Evaluation and analysis

In [5]:
exclude_O = [t for t in crf.classes_ if t != 'O']

print(classification_report(devel_y, pred_y, labels=exclude_O))
eli5.show_weights(crf, targets=sorted_for_ner(crf.classes_), top=30)

             precision    recall  f1-score   support

      B-ORG       0.83      0.80      0.81       544
      B-PER       0.88      0.77      0.82       185
      I-PER       0.85      0.90      0.88        94
      B-LOC       0.93      0.90      0.91       268
     B-DATE       0.96      0.88      0.92        51
      B-PRO       0.75      0.58      0.65       173
      I-ORG       0.78      0.69      0.73       185
     I-DATE       0.97      0.82      0.89        34
      I-PRO       0.50      0.67      0.57        82
    B-EVENT       0.00      0.00      0.00         2
    I-EVENT       0.00      0.00      0.00         2
      I-LOC       1.00      0.40      0.57        20

avg / total       0.83      0.77      0.79      1640



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


From \ To,O,B-DATE,I-DATE,B-EVENT,I-EVENT,B-LOC,I-LOC,B-ORG,I-ORG,B-PER,I-PER,B-PRO,I-PRO
O,1.079,1.39,-2.466,1.128,-1.336,1.844,-1.25,1.367,-1.806,1.103,-1.651,1.796,-1.666
B-DATE,-1.175,-0.732,4.013,0.604,-0.103,-0.047,-0.102,-0.185,-0.152,-0.556,-0.19,-0.828,-0.268
I-DATE,-1.03,-1.511,4.594,-0.063,-0.051,-0.143,-0.048,-0.225,-0.049,-0.206,-0.044,-0.49,-0.112
B-EVENT,-1.709,-0.171,-0.133,-0.049,4.16,0.46,-0.047,-0.645,-0.083,-0.546,-0.047,-0.517,-0.085
I-EVENT,-2.01,-0.114,-0.042,-0.109,4.68,0.149,-0.076,-0.717,-0.063,-0.433,-0.036,-0.383,-0.068
B-LOC,-0.773,-0.057,-0.344,1.039,-0.36,1.121,3.667,-0.579,-1.168,-1.597,-0.324,-0.667,-0.427
I-LOC,-1.512,0.129,-0.253,0.551,-0.211,0.318,3.664,-1.006,-0.303,-0.6,-0.076,-0.586,-0.1
B-ORG,-0.062,0.326,-0.586,-0.249,-0.695,0.129,-0.587,-1.114,5.372,-0.539,-0.736,0.365,-1.507
I-ORG,-0.725,-0.221,-0.269,-0.231,-0.338,-0.255,-0.313,-0.378,5.54,-1.01,-0.381,-1.258,-0.584
B-PER,-0.143,-0.299,-0.146,-0.151,-0.21,-0.667,-0.224,-1.052,-0.457,-2.486,6.34,-1.212,-0.316
